In [1]:
import time
from functools import partial
from pathlib import Path
from zipfile import ZipFile

import dask.array as da
from dask.array.core import normalize_chunks
import numpy as np
from shapely.geometry import GeometryCollection, Point
from skimage.transform import AffineTransform

from fuse.fuse import fuse_func
from utils.download_sample import download_from_dropbox
from utils.metadata import extract_coordinates, normalize_coords_to_pixel
from utils.imutils import crop_black_border, load_image, transpose, remove_background
from utils.shapely_and_napari_utils import get_transformed_array_corners, numpy_shape_to_shapely
from utils.chunks import get_chunk_coordinates, get_rect_from_chunk_boundary, find_chunk_tile_intersections

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import re 
import os
from natsort import natsorted
import enum
import napari
import warnings
warnings.filterwarnings("ignore")

# Loading own functions to sort files and extract metadata

In [3]:
@enum.unique
class Channels(enum.Enum):
    BRIGHTFIELD = 0
    GFP = 1
    RFP = 2
    IRFP = 3
    PHASE = 4
    WEIGHTS = 50
    MASK_IRFP = 96
    MASK_RFP = 97
    MASK_GFP = 98
    MASK = 99
    
def parse_filename(filename: os.PathLike) -> dict:
    """Parse an OctopusLite filename and retreive metadata from the file.

    Parameters
    ----------
    filename : PathLike
        The full path to a file to parse.

    Returns
    -------
    metadata : dict
        A dictionary containing the parsed metadata.
    """
    
    OCTOPUSLITE_FILEPATTERN =(
        "r(?P<row>[0-9]+)c(?P<column>[0-9]+)f(?P<fov>[0-9]+)p(?P<plane>[0-9]+)-ch(?P<channel>[0-9]+)"
        "sk(?P<time>[0-9]+)fk(?P<fk>[0-9]+)fl(?P<fl>[0-9]+)"
        )
    
    pth, filename = os.path.split(filename)
    params = re.match(OCTOPUSLITE_FILEPATTERN, filename)

    metadata = {
        "filename": filename,
        "channel": Channels(int(params.group("channel"))),
        "time": params.group("time"),
        "row": params.group("row"), 
        "column": params.group("column"), 
        "fov": params.group("fov"), 
        "plane": params.group("plane"), 
        "time": params.group("time"), 
        "fk": params.group("fk"), 
        "fl": params.group("fl")

    }

    return metadata

# Define parameters of mosaic compilation

In [4]:
### array that shows the location of each fov in the scan pattern
fov_scan_pattern = np.array(([2,3,4],
                             [7,6,5],
                             [8,1,9],))
fov_scan_pattern

array([[2, 3, 4],
       [7, 6, 5],
       [8, 1, 9]])

In [5]:
overlap = 0.1
chunk_size = (1080,1080)

In [6]:
image_dir = Path("/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/")

In [7]:
relevant_channels = set([parse_filename(fn)['channel'] for fn in os.listdir(image_dir)])

In [9]:
import cupy as cp

In [10]:
cp.asarray

<function cupy._creation.from_data.asarray(a, dtype=None, order=None)>

In [22]:
#iterate over channels
for ch in tqdm(Channels):#Channels:
    if ch in relevant_channels:
        print('Starting channel', ch.name)
        ### define empty z stack
        da_zt_stack = []
        ### iterate over planes 
        for p in range(1,4):
            ### define empty time series stack
            da_t_stack = []
            ### iterate over frames
            for t in (range(1,76)):
                ### get all files at that time point
                files = list(Path("/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/").glob(f"r03c03f*p0{p}-ch{ch.value}sk{t}fk*"))
                ### sort using the frame
                files.sort(key=lambda f: int(parse_filename(f)["time"]))
                ### extract tile coordinates from fn 'f' (ie. fov number)
                coords = pd.DataFrame()
                for fn in files:
                    entry = pd.DataFrame([parse_filename(fn)])
                    coords = pd.concat([coords, entry], ignore_index=True)
                ## lazy hack to make the raster scan like the provided fov_scan_pattern
                for i, row in coords.iterrows():
                    (X, Y) = tuple(map(int, np.where(fov_scan_pattern == int(row['fov']))))
                    coords.at[i, 'X'], coords.at[i, 'Y'] = X, Y
                ### lazy hack to register the row/col number as x/y location shifted by the amount of pixels in each image
                coords['um/px'] = 1/(2160*(1-overlap))
                ### turn coords into a np array of transformation amount using the um/px ratio
                normalized_coords = normalize_coords_to_pixel(coords).to_numpy()
                ### apply transforms if required -- could this be a background removal???
                input_transforms = None#[remove_background] #None #[crop_black_border, ]#transpose]
                ### define a function to load a test image and get tile shape from it
                _load_image = partial(load_image, transforms=input_transforms)
                tile_shape=_load_image(str(files[0])).shape
                ### apply the transformation to each tile to correctly mosaic them
                transforms = [AffineTransform(translation=stage_coord).params for stage_coord in normalized_coords]
                tiles = [get_transformed_array_corners(tile_shape, transform) for transform in transforms]
                ### define the bounding boxes of the tiles and overall FOV to determine the dask output shape
                all_bboxes = np.vstack(tiles)
                all_min = all_bboxes.min(axis=0)
                all_max = all_bboxes.max(axis=0)
                stitched_shape=tuple(np.ceil(all_max-all_min).astype(int))
                ### if there is a discrepancy between the top left tile and the origin then shift
                shift_to_origin = AffineTransform(translation=-all_min)
                transforms_with_shift = [t @ shift_to_origin.params for t in transforms]
                shifted_tiles = [get_transformed_array_corners(tile_shape, t) for t in transforms_with_shift]
                tiles_shifted_shapely = [numpy_shape_to_shapely(s) for s in shifted_tiles]
                ### split data into pre-defined chunks             
                chunks = normalize_chunks(chunk_size,shape=tuple(stitched_shape))
                computed_shape = np.array(list(map(sum, chunks)))
                ### check that tiles shape is correct
                assert np.all(np.array(stitched_shape) == computed_shape)
                ## get chunk details and plot as shapes
                chunk_boundaries = list(get_chunk_coordinates(stitched_shape, chunk_size))
                chunk_shapes = list(map(get_rect_from_chunk_boundary, chunk_boundaries))
                chunks_shapely = [numpy_shape_to_shapely(c) for c in chunk_shapes]
                ### iterate over files in an individual frame and attach tile info and transform
                for tile_shifted_shapely, file, transform in zip(tiles_shifted_shapely, 
                                                                 files, 
                                                                 transforms_with_shift):
                    tile_shifted_shapely.fuse_info = {'file':file, 'transform':transform}
                ### iterate over chunks for a single image and attach info
                for chunk_shapely, chunk_boundary  in zip(chunks_shapely, chunk_boundaries):
                    chunk_shapely.fuse_info = {'chunk_boundary': chunk_boundary}
                ### find intersection of tiles and chunks
                chunk_tiles = find_chunk_tile_intersections(tiles_shifted_shapely, chunks_shapely)
                ### define a fuse function to load all tiles for particular chunk
                _fuse_func=partial(fuse_func, 
                                   imload_fn=_load_image,
                                   dtype=np.uint16) 
                ### use map_blocks to calculate the fused image chunk by chunk
                target_array = da.map_blocks(func=_fuse_func,
                                             chunks=chunks, 
                                             input_tile_info=chunk_tiles,
                                             dtype=np.uint16)
                target_array = cp.asarray(target_array)
                ### append the mosaic for that particular frame to a list of mosaics
                da_t_stack.append(target_array)
            ### stack that mosaic in a time series
            da_t_stack = da.stack(da_t_stack, axis = 0)
            ### append the time series for one plane to the z-stack
            da_zt_stack.append(da_t_stack)
        ### stack the z planes together
        da_zt_stack = da.stack(da_zt_stack, axis = 0)
        ### save out as zarr
        da_zt_stack.to_zarr(f"data/zt_stack_ch{ch.value}_108tile.zarr", overwrite=True)
    else:
        print(f'Channel {ch.name} not found in image directory')

  0%|          | 0/10 [00:00<?, ?it/s]

Channel BRIGHTFIELD not found in image directory
Starting channel GFP
Processing chunk at (0, 0)
Processing chunk at (1836, 648)
Processing chunk at (4752, 2484)
Processing chunk at (2700, 1080)
Processing chunk at (2592, 4860)
Processing chunk at (3456, 5292)
Processing chunk at (1404, 3888)
Processing chunk at (108, 2700)
Processing chunk at (972, 3132)
Processing chunk at (4644, 2376)
Processing chunk at (2592, 972)
Processing chunk at (5508, 2808)
Processing chunk at (3456, 1404)
Processing chunk at (3348, 5184)
Processing chunk at (1296, 3780)
Processing chunk at (4212, 5616)
Processing chunk at (2160, 4212)
Processing chunk at (648, 2052)
Processing chunk at (540, 5832)
Processing chunk at (4320, 1296)
Processing chunk at (3780, 4968)
Processing chunk at (5184, 1728)
Processing chunk at (3132, 324)
Processing chunk at (3024, 4104)
Processing chunk at (3996, 756)
Processing chunk at (3888, 4536)
Processing chunk at (1836, 3132)
Processing chunk at (2700, 3564)
Processing chunk at 

Processing chunk at (3672, 1404)
Processing chunk at (4536, 5832)
Processing chunk at (1512, 3780)
Processing chunk at (108, 1728)
Processing chunk at (2376, 4212)
Processing chunk at (3240, 4644)
Processing chunk at (1188, 3240)Processing chunk at (2592, 0)

Processing chunk at (4860, 2484)Processing chunk at (3456, 432)
Processing chunk at (5724, 2916)
Processing chunk at (1296, 2808)
Processing chunk at (3564, 5292)
Processing chunk at (4536, 1944)

Processing chunk at (4428, 5724)Processing chunk at (3348, 324)
Processing chunk at (4212, 756)Processing chunk at (2052, 3132)
Processing chunk at (0, 1728)
Processing chunk at (2916, 3564)


Processing chunk at (756, 5940)Processing chunk at (864, 2160)
Processing chunk at (3780, 3996)

Processing chunk at (1728, 2592)Processing chunk at (5400, 1836)

Processing chunk at (432, 5400)Processing chunk at (4104, 4644)
Processing chunk at (4968, 5076)

Processing chunk at (540, 972)
Processing chunk at (2808, 3456)
Processing chunk at (3780

Processing chunk at (2268, 3240)
Processing chunk at (5292, 1296)
Processing chunk at (5184, 5076)
Processing chunk at (3132, 3672)
Processing chunk at (3996, 4104)
Processing chunk at (540, 5292)Processing chunk at (5076, 4968)

Processing chunk at (756, 1080)
Processing chunk at (5940, 5400)
Processing chunk at (1620, 1512)Processing chunk at (3888, 3996)

Processing chunk at (4752, 4428)Processing chunk at (432, 540)

Processing chunk at (324, 4320)Processing chunk at (4968, 216)Processing chunk at (1404, 5832)


Processing chunk at (1512, 1404)Processing chunk at (2376, 1836)

Processing chunk at (216, 4212)
Processing chunk at (324, 432)
Processing chunk at (3240, 2268)
Processing chunk at (1188, 864)Processing chunk at (1080, 4644)

Processing chunk at (1944, 5076)
Processing chunk at (4860, 108)Processing chunk at (5724, 540)
Processing chunk at (5616, 4320)

Processing chunk at (3564, 2916)Processing chunk at (4428, 3348)Processing chunk at (5292, 3780)

Processing chunk at (22

Processing chunk at (1296, 2268)Processing chunk at (4212, 4104)
Processing chunk at (2160, 2700)

Processing chunk at (0, 5076)Processing chunk at (864, 5508)

Processing chunk at (1728, 5940)

Processing chunk at (5400, 5184)

Processing chunk at (648, 540)Processing chunk at (540, 4320)
Processing chunk at (3780, 3456)

Processing chunk at (5184, 216)Processing chunk at (3024, 2592)

Processing chunk at (3888, 3024)Processing chunk at (1836, 1620)Processing chunk at (2700, 2052)


Processing chunk at (4968, 4536)
Processing chunk at (540, 432)Processing chunk at (5832, 4968)Processing chunk at (1404, 4860)

Processing chunk at (5076, 108)
Processing chunk at (108, 3672)
Processing chunk at (5940, 540)

Processing chunk at (972, 4104)
Processing chunk at (4644, 3348)
Processing chunk at (2592, 1944)
Processing chunk at (5508, 3780)
Processing chunk at (3456, 2376)
Processing chunk at (1404, 972)
Processing chunk at (1296, 4752)Processing chunk at (2160, 5184)
Processing chunk at (194

...

Processing chunk at (864, 4104)Processing chunk at (1836, 756)

Processing chunk at (1728, 4536)Processing chunk at (5508, 0)

Processing chunk at (540, 3564)
Processing chunk at (4212, 2808)
Processing chunk at (3024, 1836)
Processing chunk at (2916, 5616)
Processing chunk at (3888, 2268)Processing chunk at (1728, 648)

Processing chunk at (2592, 1080)
Processing chunk at (432, 3456)Processing chunk at (1296, 3888)

Processing chunk at (4968, 3132)
Processing chunk at (2916, 1728)
Processing chunk at (5832, 3564)
Processing chunk at (3780, 2160)Processing chunk at (3672, 5940)

Processing chunk at (4644, 2592)
Processing chunk at (2484, 4968)Processing chunk at (3348, 5400)
Processing chunk at (2268, 0)

Processing chunk at (2160, 3780)
Processing chunk at (3132, 432)
Processing chunk at (108, 2376)Processing chunk at (5400, 2916)

Processing chunk at (972, 2808)
Processing chunk at (3240, 5292)
Processing chunk at (1836, 3240)Processing chunk at (4104, 5724)
Processing chunk at (3456

Processing chunk at (2484, 1512)
Processing chunk at (4752, 3996)Processing chunk at (5616, 4428)
Processing chunk at (5292, 3240)

Processing chunk at (3240, 1836)

Processing chunk at (1188, 432)
Processing chunk at (4104, 2268)Processing chunk at (2052, 864)

Processing chunk at (1944, 4644)
Processing chunk at (2808, 5076)
Processing chunk at (5724, 108)
Processing chunk at (756, 3672)
Processing chunk at (1620, 4104)
Processing chunk at (324, 2916)
Processing chunk at (3996, 2160)Processing chunk at (4860, 2592)
Processing chunk at (2808, 1188)Processing chunk at (2700, 4968)

Processing chunk at (648, 3564)
Processing chunk at (3564, 5400)
Processing chunk at (1512, 3996)
Processing chunk at (4428, 5832)
Processing chunk at (2916, 756)Processing chunk at (2376, 4428)


Processing chunk at (0, 1836)Processing chunk at (3672, 1080)
Processing chunk at (4536, 1512)
Processing chunk at (2484, 108)Processing chunk at (5400, 1944)
Processing chunk at (3348, 540)Processing chunk at (324

Processing chunk at (648, 2592)
Processing chunk at (5940, 3132)
Processing chunk at (2916, 5076)
Processing chunk at (1512, 3024)
Processing chunk at (3780, 5508)
Processing chunk at (4644, 5940)
Processing chunk at (2700, 108)
Processing chunk at (3564, 540)Processing chunk at (4428, 972)
Processing chunk at (1404, 2916)
Processing chunk at (2268, 3348)Processing chunk at (3132, 3780)

Processing chunk at (216, 1944)Processing chunk at (108, 5724)Processing chunk at (1080, 2376)

Processing chunk at (4752, 1620)
Processing chunk at (5616, 2052)Processing chunk at (5508, 5832)
Processing chunk at (3456, 4428)



Processing chunk at (4320, 4860)
Processing chunk at (2160, 3240)Processing chunk at (5184, 5292)
Processing chunk at (1836, 2700)

Processing chunk at (5508, 1944)Processing chunk at (540, 5508)Processing chunk at (1944, 2268)

Processing chunk at (4212, 4752)

Processing chunk at (5184, 1404)
Processing chunk at (5076, 5184)Processing chunk at (756, 1296)

Processing chunk a

Processing chunk at (4644, 1080)Processing chunk at (4536, 4860)

Processing chunk at (2484, 3456)
Processing chunk at (5400, 5292)
Processing chunk at (3348, 3888)Processing chunk at (5724, 5940)Processing chunk at (2160, 2268)


Processing chunk at (108, 864)Processing chunk at (5400, 1404)
Processing chunk at (5292, 5184)
Processing chunk at (972, 1296)
Processing chunk at (864, 5076)Processing chunk at (1836, 1728)
Processing chunk at (4104, 4212)
Processing chunk at (5508, 972)
Processing chunk at (4320, 0)

Processing chunk at (5184, 432)Processing chunk at (3024, 2808)


Processing chunk at (864, 1188)Processing chunk at (1728, 1620)
Processing chunk at (2592, 2052)

Processing chunk at (540, 648)
Processing chunk at (432, 4428)
Processing chunk at (1296, 4860)
Processing chunk at (5076, 324)
Processing chunk at (4968, 4104)Processing chunk at (5940, 756)

Processing chunk at (5832, 4536)Processing chunk at (2916, 2700)

Processing chunk at (3780, 3132)
Processing chunk at (4644


KeyboardInterrupt



In [12]:
da_stack_gfp = da.from_zarr(f"zt_stack_ch{1}.zarr")
da_stack_rfp = da.from_zarr(f"zt_stack_ch{2}.zarr")
v = napari.Viewer()
v.add_image(da_stack_gfp, name="gfp", contrast_limits = [0,2352], blending = 'additive', colormap= 'green')# colormap = 'g')
v.add_image(da_stack_rfp, name="rfp", contrast_limits = [103,164], blending = 'additive', colormap = 'magenta')

<Image layer 'rfp' at 0x7fdf4fbc3e80>

### Testing GPU capacity

In [13]:
da_stack_gfp = da.from_zarr(f"data/zt_stack_ch{1}.zarr")


In [17]:
da_stack_gfp_gpu = da_stack_gfp.map_blocks(cp.asarray)
da_stack_gfp_gpu

dask.array<asarray, shape=(3, 75, 6048, 6048), dtype=uint16, chunksize=(1, 1, 2160, 2160), chunktype=cupy.ndarray>

In [18]:
da_stack_gfp

dask.array<from-zarr, shape=(3, 75, 6048, 6048), dtype=uint16, chunksize=(1, 1, 2160, 2160), chunktype=numpy.ndarray>

#### Would then need a reason to run an image manipulation on this ``dask_stack_gfp_gpu`` 